# 0 - Fill missing values

- import data
- create custom metrics - order
- fill in rows (by order)
- export as pickle

In [2]:
import numpy as np
import pandas as pd

In [10]:
def get_timestamp_order(ts):
    '''timestamp to order'''
    ts = ts.split(':')
    return int(ts[0])*4 + int(ts[1])//15

def timestamp_to_hour_minute(timestamp):
    '''timestamp to hour, minute'''
    timestamp = timestamp.split(':')
    return int(timestamp[0]), int(timestamp[1])

def format_order(row):
    '''get order'''
    return row['day']*96 + get_timestamp_order(row['timestamp'])

def order_to_time(order):
    '''order to hour, minute'''
    order = order % 96
    return order//4, order%4 * 15

In [4]:
df = pd.read_csv('../data/training.csv')

In [6]:
df['order'] = df.apply(format_order, axis=1)

In [11]:
formatted_dict = {}
GEOHASH_LIST = df.geohash6.unique()
MIN_ORDER = np.min(df.order.unique())
MAX_ORDER = np.max(df.order.unique())

for i, gh in enumerate(GEOHASH_LIST):
    temp_df = df[df.geohash6 == gh]
    temp_df.set_index('order', inplace=True)
    temp_dict = temp_df.to_dict('index')
    for order in range(MIN_ORDER, MAX_ORDER+1):
        hour, minute = order_to_time(order)
        day = order // 96
        if order not in temp_dict:
            demand = 0.0
        else:
            demand = temp_dict[order]['demand']
        formatted_dict[order + i*MAX_ORDER] = {
            'order': order,
            'geohash6': gh,
            'timestamp': '{0}:{1}'.format(hour, minute),
            'day': day,
            'demand': demand,
        }
        
df = pd.DataFrame.from_dict(formatted_dict, orient='index')
df.to_pickle('../data/0-df.pkl')

In [23]:
test_df = df[(df.day > 50) & (df.geohash6 == 'qp03wc')]
test_df.to_csv('../data/test.csv', index=False)